In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Case of study 1

Information about number of users, from:
https://www.freeletics.com/en/press/wp-content/uploads/sites/24/2015/02/EN-Presskit-Febraury-2017_web.pdf



In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as grds
import scipy 
from scipy import signal as signal

%matplotlib inline

### Choose meaningful numbers for X,Y,Z.

The parameter X seems to be the arrival rate of users to the application. Because the total amount of Freeletics users is around 14 million of users. The estimation of having half of them having access to the feature in the same day seems reasonable.

The parameter Y will model loading time without the feature. Because we want to have the servers working in a proper range of occupancy and user waiting time it is a good idea to fix the desired traffic intensity $\rho$. If this factor is larger than one we could not provide service and if it is too small we are overstimating the users activity in our servers. Hence a value of $\rho=0.8=\frac{\lambda}{\mu}$ seems a good choice. Finally $Y=\frac{1}{\mu}$

The parameter Z is the impact of the new feature in the model. Because we do not have any prior about it, I decided to model it as a normal distributed random variable with zero mean and standard deviation being a tenth of the loading time that the system have without the feature.

In [3]:
# Defining the parameters of our model based on the data sources explained before
# This section cover the question:
# Choose meaningful numbers for X,Y,Z.

Total_Customers = 14e6
DailyCustomers = Total_Customers * 0.5 #Assuming half of the customers acces the application

# The parameter X is related with the traffic incoming to the application server. It is the incoming rate
X= DailyCustomers/ 24 / 3600 #users arrived per time unit (seconds)
lambd = X

# The parameter Y is related with the loading time of the service without the new feature.
# Because we want to have a proper use of the server we assume traffic intensity to be 0.8
rho=0.8
mu = lambd/rho

Y= 1.0/mu # ms per user

# The parameter Z is related to the influence of the new feature in the server.
#It will be modeled as a normal distribution with mean=0 and std = Y*0.1.
# This assumes that the load of the feature will deviate one tenth of the original waiting time

Z= np.random.normal(loc=0,scale=Y*0.1,size=1) # ms per user

Beta = Y-Z

print 'lambda=X=',X,' users/second'
print 'mu=',1.0/Y,' users/second'
print 'Y=',Y,' seconds'
print 'Z=',Z,'seconds'
print 'Beta=',Beta,'seconds'
print '1/Beta=',1.0/Beta,'users/seconds'

lambda=X= 81.0185185185  users/second
mu= 101.273148148  users/second
Y= 0.00987428571429  seconds
Z= [ 0.00122791] seconds
Beta= [ 0.00864637] seconds
1/Beta= [ 115.65545436] users/seconds


### Generate and visualize dummy data that models the scenario above over 24 hours. How did you model user traffic and SFLT? What parameters did you use? Why?

The user traffic can be modeled as an homogeneus Poisson process. This choice makes sense in an arrival situation in which the arrival is memory less, that is there is independency of the events that arrived in the past to the system.Despite of this unrealistic situation, this assumption provides a big mathematical background for estimations and modeling of the whole system. The probability of having a given of arrivals is given by the Poisson distribution: 

$ Traffic = \frac{(\lambda t^n)}{n!} \exp(-\lambda t)$

where the parameter $\lambda$ is excatly the parameter X, i.e. the users (or arrival of users) per day. It can be checked that if the arrival process follows a Poisson distribution then the time between sucessive arrivals follows an exponential distribution with same paremeter as the Poisson distribution $\lambda$.

In the other hand, the capacity of the server to process the users requests will follow an exponential distribution with parameter $\mu$

The fact of having an poissonal arrival process and an exponential loading (serving) time together with a policy of FCFS (first-come, first-served) makes possible to model the whole system as a M/M/1 queue system. The advantages of this system, is that despite of its simplcity can provide a good estimator for measuring the impact of the new feature in the system.

Under this assumptions the SFLT will follow an exponential distribution of scale factor Beta = Y-Z.

$ SFLT = \frac{1}{Y-Z} \exp(-\frac{1}{Y-Z} t)$



In [4]:
# Generating the distributions of the data
T = np.arange(24*3600)

print 'T',T



Arrival_process = np.random.poisson(lambd, size=(1,len(T))) # Poisson process with lambda value 
Traffic =Arrival_process[0]

SFLT = np.random.exponential(scale=Beta, size=(1,len(T)))


print 'Traffic',Traffic
print 'SFLT',SFLT

print 'E[SFLT]',np.mean(SFLT[0])



# Definition of the figure.
figid = plt.figure(1,figsize=(20, 10), dpi=180)
# Definition of the matrix of plots. In this case the situation is more complex that is why I need to define a
# matrix. It will be a dim[2x3] matrix.
gs = grds.GridSpec(2,3)
ax1 = plt.subplot(gs[0, :])
ax2 = plt.subplot(gs[1,:])

ax1.plot(T,Traffic,color='blue')
ax2.plot(T,SFLT[0],color='orange')

# Setting the xlabel with the desired parameters
override = {
    'fontsize'            : '14',
    'verticalalignment'   : 'top',
    'horizontalalignment' : 'center'
    }
ax1.set_xlabel('t[seconds]',override)
ax2.set_xlabel('t[seconds]',override)

ax1.set_ylabel('Incoming users',override)
ax2.set_ylabel('SFLT [seconds]',override)    

T [    0     1     2 ..., 86397 86398 86399]
Traffic [65 83 79 ..., 85 87 77]
SFLT [[ 0.01398434  0.02442391  0.00333444 ...,  0.01015792  0.0256075
   0.00609031]]
E[SFLT] 0.00867698512491


AttributeError: 'module' object has no attribute 'to_rgba'

### Given the dummy data, formulate a hypothesis, to answer the question of “is the feature affecting the SFLT in any way?

To check whether the feature is affecting the SFLT in any way it will be neccessary to set a null hypothesis $H_0$ and an alternative hypothesis $H_1$. The fact "the new feature is affecting the SFLT in any way" can be translated into 'the new feature is NOT affecting the SFLT in any way', this will mean that if it is not affecting the SFLT the mean SFLT should remain equal. This set the null hypothesis in the next way:

$H_0:$ the mean of the SFLT with the new feature and the mean of the SFLT without the new feature are the same.

and hence the alternative hypothesis will be:

$H_1:$ the mean of the SFLT with the new feature and the mean of the SFLT without the new feature are different, and hence the new feature has an impact on the service.

### Name, setup and explain the test you would perform to test this hypothesis.

The most suitable statistical test in my opinion to test this hypothesis is the <b>Welch's t-test</b>. The test is a variation of the two-sample t-test; it is more robust by not assuming equal variance between the two populations from where we extract the sample mean. 

For using this test I will set the significance level at 0.05



### Implement the set of formulas that you would need to validate the hypothesis.

The set of formulas to validate this hypothesis are:

$\Huge t^*=\frac{E[SFLT_0]-E[SFLT_1]}{\sqrt{\frac{s_{0}^2}{N_0}+\frac{s_{1}^2}{N_1}}}$

where:

$E[SFLT_n]$ are the sample mean of each population

$N_n$ are the sample size of each population

$s_n$ are the standard deviation of each population

In [5]:
#Implementation of the Welch's t-test


SFLT_0 = np.random.exponential(scale=Y, size=(1,len(T))) #Sampled process for 24 hours without the new feature
SFLT_1 = np.random.exponential(scale=Beta, size=(1,len(T))) #Sampled process for 24 with the new feature

# Computing the important parameters of the sampled process
E_SFLT_0 = np.mean(SFLT_0)
E_SFLT_1 = np.mean(SFLT_1)

std_SFLT_0 = np.std(SFLT_0)
std_SFLT_1 = np.std(SFLT_1)

N_0=len(T)
N_1=N_0



print 'E[SFLT_0]',E_SFLT_0
print 'E[SFLT_1]',E_SFLT_1

print 's_0', std_SFLT_0
print 's_1', std_SFLT_1

print 'N_0',N_0
print 'N_1',N_1


t_star = (E_SFLT_0-E_SFLT_1)/(np.sqrt(((std_SFLT_0**2)/N_0)+((std_SFLT_1**2)/N_1)))

ddof_num = ((((std_SFLT_0**2)/N_0)+((std_SFLT_1**2)/N_1))**2)
ddof_denom = ((std_SFLT_0**4)/((N_0**2)*(N_0-1)))+((std_SFLT_1**4)/((N_1**2)*(N_1-1)))
Degrees_of_freedom = ddof_num/ddof_denom


print 't_star',t_star
print 'Degrees of freedom',Degrees_of_freedom

s = np.random.standard_t(Degrees_of_freedom,1e6)



if np.sum((s<=t_star).astype(int))<0.05:
    print '************************'
    print 'NULL HYPOTHESIS REJECTED'
    print '************************'
else:
    print '****************************'
    print 'NULL HYPOTHESIS NOT REJECTED'
    print '****************************'

#Using the inbuild Welch test for validation.
print scipy.stats.ttest_ind(SFLT_0.T,SFLT_1.T,equal_var=False)


# To have a visual reference
plt.figure()
plt.hist(s)
plt.plot([t_star,t_star],[0,5e5],'r')


E[SFLT_0] 0.00986631273601
E[SFLT_1] 0.00866427666406
s_0 0.00988416640626
s_1 0.00869044764773
N_0 86400
N_1 86400
t_star 26.8456899552
Degrees of freedom 170012.360418
****************************
NULL HYPOTHESIS NOT REJECTED
****************************
Ttest_indResult(statistic=array([ 26.8455346]), pvalue=array([  2.04114937e-158]))


/home/javier/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


AttributeError: 'module' object has no attribute 'to_rgba'

### Should we roll back this feature? Why? Why not? Please provide a set of numbers that you used for this decision.

Depending on the result on the previous test:

If the null hypothesis is REJECTED: Rejecting the null hypothesis means that the at a 5% level of significance the data provide evidence that the new feature have an impact on the SFLT. The suggestion from a perspective of bussines intelligence department is to wait to release this feature until we have a better estimation of performance of the servers to provide better QoS to the users. A loading time is a critical value for the customer experience and any delay will generate people dropping not only this service but the whole freeletics application.

If the null hypothesis is NOT REJECTED: means that the new feature do not have an inmpact on the SFLT and hence we are serving the customers at the same level as before.

### Provide a visualization that shows SFLT over time [hours] with appropriate confidence intervals.

To provide the visualization of the SFLT with the new feature over time with confidence intervals the first thing to do is to bin the given process in one hour bin and compute the mean of that period of time and the confidence interval.

The code below summarize the procedure.

In [6]:
Hours = 24
Seconds = 3600
z_critical= scipy.stats.expon.ppf(q=0.99,scale=Beta)

mean_SFLT_1=[]
std_SFLT_1=[]
margin_of_error=[]
del confidence_interval
for i in range(Hours):
    local_SFLT = SFLT_1.T[Seconds*i:Seconds*(i+1)]
    mean_SFLT_1.append(np.mean(local_SFLT))
    std_SFLT_1.append(np.std(local_SFLT))
    margin_of_error.append(z_critical[0] * (np.std(local_SFLT)/math.sqrt(Seconds)))
    

mean_SFLT_1 = np.array(mean_SFLT_1)
std_SFLT_1 = np.array(std_SFLT_1)
margin_of_error = np.array(margin_of_error)

confidence_interval = (mean_SFLT_1 - margin_of_error,
                           mean_SFLT_1 + margin_of_error)  
    
# Definition of the figure.
figid = plt.figure(1,figsize=(20, 10), dpi=180)
# Definition of the matrix of plots. In this case the situation is more complex that is why I need to define a
# matrix. It will be a dim[2x3] matrix.
gs = grds.GridSpec(1,1)
ax1 = plt.subplot(gs[0, :])

ax1.plot(mean_SFLT_1)
ax1.plot(mean_SFLT_1,'r.')
ax1.plot(mean_SFLT_1-margin_of_error,'r')
ax1.plot(mean_SFLT_1+margin_of_error,'r')

# Setting the xlabel with the desired parameters
override = {
    'fontsize'            : '14',
    'verticalalignment'   : 'top',
    'horizontalalignment' : 'center'
    }
ax1.set_xlabel('t[hours]',override)
ax1.set_ylabel('SFLT[seconds]')

NameError: name 'confidence_interval' is not defined

### Optional: is there a correlation between traffic and SFLT? How would you prove it? What would that indicate?

Calculating the correlation coefficient of the two signals (cell below), Traffic and SFLT, it is possible to see that the correlation coefficient is pretty low. This means that the two signals are highly uncorrelated (independent) and indicates that we are far from reaching the state in which the waiting time is driven by the arrivals of the customer (that is having a $\rho \approxeq 1$ ). For values like the one used here of $\rho = 0.8$ the system is still in a state in which the arrivals can be easily managed by the server and hence the delays fall whithin the the distribution of the own capabilities of the server rather than on the distribution of the load (arrivals) of the server.

In [292]:

print 'Correlation coefficient:',np.corrcoef(Traffic,SFLT_1[0,:])[1,0]

Correlation coefficient: 0.00125025545461
